In [3]:
from catboost.datasets import amazon
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import os
np.set_printoptions(precision=4)
!jupyter nbextension enable --py widgetsnbextension

train_df, test_df = amazon()

train_df.head()

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
y = train_df.ACTION
X = train_df.drop("ACTION", axis=1)

In [5]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [6]:
X.shape

(32769, 9)

In [7]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100)
model.fit(X, y, cat_features=cat_features, verbose=10)

Learning rate set to 0.377604
0:	learn: 0.4528598	total: 504ms	remaining: 49.9s
10:	learn: 0.1744186	total: 1.53s	remaining: 12.4s
20:	learn: 0.1676119	total: 2.67s	remaining: 10s
30:	learn: 0.1652446	total: 3.85s	remaining: 8.56s
40:	learn: 0.1633644	total: 5.01s	remaining: 7.21s
50:	learn: 0.1621892	total: 6.2s	remaining: 5.95s
60:	learn: 0.1609164	total: 7.34s	remaining: 4.69s
70:	learn: 0.1594572	total: 8.52s	remaining: 3.48s
80:	learn: 0.1585876	total: 9.68s	remaining: 2.27s
90:	learn: 0.1573593	total: 10.8s	remaining: 1.07s
99:	learn: 0.1566977	total: 11.9s	remaining: 0us


In [8]:
model.predict_proba(X)

array([[0.0098, 0.9902],
       [0.0101, 0.9899],
       [0.0579, 0.9421],
       ...,
       [0.0118, 0.9882],
       [0.1891, 0.8109],
       [0.0235, 0.9765]])

In [9]:
from catboost import Pool
pool = Pool(data=X, label=y, cat_features=cat_features)

In [10]:
from sklearn.model_selection import train_test_split

data = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(
    data=X_train, 
    label=y_train, 
    cat_features=cat_features
)

validation_pool = Pool(
    data=X_validation, 
    label=y_validation, 
    cat_features=cat_features
)

In [11]:
model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1
)
model.fit(train_pool, eval_set = validation_pool, verbose=False)

In [12]:
model = CatBoostClassifier(
    iterations=15
)
model.fit(train_pool, eval_set = validation_pool)

Learning rate set to 0.441257
0:	learn: 0.4226231	test: 0.4217069	best: 0.4217069 (0)	total: 108ms	remaining: 1.51s
1:	learn: 0.3157972	test: 0.3136469	best: 0.3136469 (1)	total: 243ms	remaining: 1.58s
2:	learn: 0.2631196	test: 0.2603395	best: 0.2603395 (2)	total: 359ms	remaining: 1.44s
3:	learn: 0.2334650	test: 0.2294580	best: 0.2294580 (3)	total: 470ms	remaining: 1.29s
4:	learn: 0.2077060	test: 0.2017327	best: 0.2017327 (4)	total: 586ms	remaining: 1.17s
5:	learn: 0.1961364	test: 0.1883112	best: 0.1883112 (5)	total: 705ms	remaining: 1.06s
6:	learn: 0.1879266	test: 0.1794018	best: 0.1794018 (6)	total: 815ms	remaining: 931ms
7:	learn: 0.1841218	test: 0.1743149	best: 0.1743149 (7)	total: 885ms	remaining: 775ms
8:	learn: 0.1814626	test: 0.1698731	best: 0.1698731 (8)	total: 998ms	remaining: 665ms
9:	learn: 0.1785403	test: 0.1650335	best: 0.1650335 (9)	total: 1.1s	remaining: 550ms
10:	learn: 0.1771678	test: 0.1634002	best: 0.1634002 (10)	total: 1.21s	remaining: 439ms
11:	learn: 0.1762101	te

In [13]:
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    custom_loss=['AUC', 'Accuracy']
)

model.fit(
    train_pool, 
    eval_set = validation_pool, 
    verbose=False,
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [14]:
model1 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=100,
    train_dir='learning_rate_0.7'
)

model2 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    train_dir='learning_rate_0.01'
)

model1.fit(train_pool, eval_set=validation_pool, verbose=20)
model2.fit(train_pool, eval_set=validation_pool, verbose=20)

0:	learn: 0.3264513	test: 0.3248170	best: 0.3248170 (0)	total: 132ms	remaining: 13s
20:	learn: 0.1688825	test: 0.1574182	best: 0.1573949 (16)	total: 2.22s	remaining: 8.36s
40:	learn: 0.1632884	test: 0.1582531	best: 0.1571533 (23)	total: 4.38s	remaining: 6.31s
60:	learn: 0.1584388	test: 0.1573279	best: 0.1569712 (52)	total: 6.5s	remaining: 4.16s
80:	learn: 0.1555790	test: 0.1582746	best: 0.1569712 (52)	total: 8.68s	remaining: 2.04s
99:	learn: 0.1519826	test: 0.1590949	best: 0.1569712 (52)	total: 10.7s	remaining: 0us

bestTest = 0.1569712214
bestIteration = 52

Shrink model to first 53 iterations.
0:	learn: 0.6853769	test: 0.6853610	best: 0.6853610 (0)	total: 147ms	remaining: 14.6s
20:	learn: 0.5575578	test: 0.5568257	best: 0.5568257 (20)	total: 1.75s	remaining: 6.58s
40:	learn: 0.4678112	test: 0.4663769	best: 0.4663769 (40)	total: 3.4s	remaining: 4.89s
60:	learn: 0.4029225	test: 0.4011544	best: 0.4011544 (60)	total: 4.83s	remaining: 3.09s
80:	learn: 0.3551621	test: 0.3530433	best: 0.353

In [15]:
from catboost import MetricVisualizer
MetricVisualizer(['learning_rate_0.7', 'learning_rate_0.01']).start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [16]:
# Cross-validation
from catboost import cv

params = {
    'loss_function': 'Logloss',
    'iterations': 80,
    'custom_loss': 'AUC',
    'learning_rate': 0.5
}

cv_data = cv(
    params = params,
    pool = train_pool,
    fold_count = 5,
    shuffle = True,
    partition_random_seed = 0,
    plot = True,
    verbose = False    
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.1631730144
bestIteration = 21

Training on fold [1/5]

bestTest = 0.1608272543
bestIteration = 77

Training on fold [2/5]

bestTest = 0.1694535356
bestIteration = 12

Training on fold [3/5]

bestTest = 0.1559891322
bestIteration = 41

Training on fold [4/5]

bestTest = 0.1644437541
bestIteration = 30



In [17]:
cv_data.head(10)

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-AUC-mean,test-AUC-std
0,0,0.305831,0.000020,0.305810,0.000014,0.496175,0.014042
1,1,0.234755,0.001020,0.235427,0.000274,0.569789,0.030202
2,2,0.196036,0.002694,0.202469,0.003171,0.761543,0.025878
3,3,0.182656,0.001401,0.191400,0.001531,0.801902,0.008434
4,4,0.175531,0.001515,0.185466,0.002237,0.817244,0.008916
5,5,0.172046,0.002150,0.182478,0.002121,0.823333,0.006377
6,6,0.170193,0.001849,0.180450,0.002460,0.828977,0.007246
7,7,0.168321,0.002187,0.178140,0.002177,0.834106,0.010527
8,8,0.167416,0.002498,0.176438,0.002126,0.835628,0.011008
9,9,0.166474,0.003010,0.175041,0.002259,0.837496,0.011698


In [18]:
# Overfitting Detector

model_with_early_stop = CatBoostClassifier(
    iterations=200,
    learning_rate=0.5,
    early_stopping_rounds=20
)

model_with_early_stop.fit(
    train_pool, 
    eval_set = validation_pool, 
    verbose=False,
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [19]:
# Prediction
model = CatBoostClassifier(iterations=200, learning_rate=0.3)
model.fit(train_pool, verbose=50)

0:	learn: 0.4076708	total: 230ms	remaining: 45.8s
50:	learn: 0.1468942	total: 12.7s	remaining: 37.2s
100:	learn: 0.1341168	total: 25.4s	remaining: 24.9s
150:	learn: 0.1242725	total: 38.3s	remaining: 12.4s
199:	learn: 0.1148999	total: 50.8s	remaining: 0us


In [20]:
print(model.predict(X_validation))

[1 1 1 ... 1 1 1]


In [21]:
print(model.predict_proba(X_validation))

[[0.0065 0.9935]
 [0.0233 0.9767]
 [0.0042 0.9958]
 ...
 [0.0224 0.9776]
 [0.0722 0.9278]
 [0.0092 0.9908]]


In [22]:
raw_pred = model.predict(
    X_validation,
    prediction_type='RawFormulaVal'
    
)

print(raw_pred)

[5.027  3.7375 5.4608 ... 3.7768 2.5538 4.6809]


In [23]:
# Feature Importances
np.array(model.get_feature_importance(prettified=True))

array([['RESOURCE', 17.382551195490088],
       ['ROLE_DEPTNAME', 17.35037716501782],
       ['MGR_ID', 14.984907833403316],
       ['ROLE_ROLLUP_2', 11.909655586316477],
       ['ROLE_FAMILY_DESC', 9.761536267864287],
       ['ROLE_TITLE', 9.638469280582443],
       ['ROLE_FAMILY', 7.370120955656141],
       ['ROLE_ROLLUP_1', 6.816752235258708],
       ['ROLE_CODE', 4.785629480410685]], dtype=object)

In [24]:
np.array(model.get_feature_importance(
    train_pool,
    'LossFunctionChange',
    prettified=True
))

array([['RESOURCE', 0.022919717773406196],
       ['MGR_ID', 0.016746814887440925],
       ['ROLE_DEPTNAME', 0.007538196263116691],
       ['ROLE_TITLE', 0.006690289660723584],
       ['ROLE_FAMILY_DESC', 0.003517874912121435],
       ['ROLE_ROLLUP_2', 0.003098796585952841],
       ['ROLE_ROLLUP_1', 0.0011968922847948938],
       ['ROLE_CODE', 0.000717905366603625],
       ['ROLE_FAMILY', -0.0007305941438350746]], dtype=object)

In [25]:
# Shap Values
print(model.predict_proba([X.iloc[1,:]]))
print(model.predict_proba([X.iloc[91,:]]))

[[0.0089 0.9911]]
[[0.9881 0.0119]]


In [27]:
shap_values = model.get_feature_importance(
    validation_pool,
    'ShapValues'
)
expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]
print(shap_values.shape)

(6554, 9)


In [32]:
# Snapshotting
model = CatBoostClassifier(
    iterations=100,
    save_snapshot=True,
    snapshot_file='snapshot.bkp',
    snapshot_interval=1
)

model.fit(train_pool, eval_set=validation_pool, verbose=10)

Learning rate set to 0.193326

bestTest = 0.1573067324
bestIteration = 81

Shrink model to first 82 iterations.


In [37]:
# Hyperparameter tuning
tunned_model = model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    bagging_temperature=1
)

tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    verbose=False,
    eval_set=(X_validation, y_validation),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))